# Universidad del Valle de Guatemala
## Security Data Science - 10
* Juan Manuel Marroquin Alfaro - 19845
* Carlos Alberto Raxtum Ramos - 19721
* Jose Abraham Gutierrez Corado - 19111
* Walter Danilo Saldaña Salguero - 19897
* Javier Alejandro Cotto Argueta - 19324

## Proyecto Fase 2: Selección de caracteristicas y limpieza de datos

In [10]:
from PIL import Image
import numpy as np
import sys
import os
import csv# default format can be changed as needed
def createFileList(myDir, format='.pgm'):
    fileList = []
    print(myDir)
    labels = []
    names = []
    keywords = {"K" : "1",} # keys and values to be changed as needed
    for root, dirs, files in os.walk(myDir, topdown=True):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
            for keyword in keywords:
                if keyword in name:
                    labels.append(keywords[keyword])
                else:
                    continue
            names.append(name)
    return fileList, labels, names

myFileList, labels, names  = createFileList('./BOSSbase/BOSSbase_1.01')
print(labels)
i = 0
for file in myFileList:
    print(file)
    img_file = Image.open(file)
    # img_file.show()# get original image parameters...
    width, height = img_file.size
    format = img_file.format
    mode = img_file.mode# Make image Greyscale
    img_grey = img_file.convert('L')
    #img_grey.save('result.png')
    #img_grey.show()# Save Greyscale values
    value = np.asarray(img_grey.getdata(), dtype=int).reshape((width, height))
    value = value.flatten()
    
    value = np.append(value,labels[i])
    i +=1
    
    print(value)
    with open("images-db.csv", 'a') as f:
        writer = csv.writer(f)
        writer.writerow(value)

./BOSSbase/BOSSbase_1.01
[]
./BOSSbase/BOSSbase_1.01\1.pgm


IndexError: list index out of range

Importamos librerias

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
import itertools
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import SilhouetteVisualizer
from os import listdir
from os.path import isfile, join
import os

## DCTR Dataset

In [3]:
df = pd.read_csv('dctr_cover.csv', header = None)
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,7990,7991,7992,7993,7994,7995,7996,7997,7998,7999
5590,0.000000,0,0.000000,0,1.00000,0.000000,0.000000,0.000000,0.000000,1.00000,...,0.98904,0.010456,0.000504,0.000000,0.000000,0.98841,0.011086,0.000504,0.000000,0.000000
3107,0.003906,0,0.008789,0,0.98730,0.002108,0.002232,0.004836,0.003348,0.98748,...,0.88133,0.091333,0.020408,0.004787,0.002142,0.87150,0.098765,0.021668,0.005543,0.002520
6659,0.000244,0,0.000244,0,0.99951,0.000124,0.000248,0.000000,0.000000,0.99963,...,0.98967,0.010078,0.000252,0.000000,0.000000,0.98614,0.013605,0.000252,0.000000,0.000000
2256,0.000000,0,0.000000,0,1.00000,0.000000,0.000000,0.000000,0.000000,1.00000,...,0.99975,0.000252,0.000000,0.000000,0.000000,0.99950,0.000504,0.000000,0.000000,0.000000
2949,0.002197,0,0.006836,0,0.99097,0.001364,0.003224,0.003348,0.002976,0.98909,...,0.75888,0.178000,0.046989,0.010078,0.006047,0.74326,0.185940,0.050391,0.013102,0.007307


DCTR es una técnica aplicada a los píxeles de la imagen en el dominio espacial para transformarlos a un dominio de frecuencias númericas, para facilitar la utilización y manipulación de este.

## GFR Dataset

In [8]:
df2 = pd.read_csv('gfr_cover.csv', header = None)
df2.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,16990,16991,16992,16993,16994,16995,16996,16997,16998,16999
5674,1,0,0,0,0,1,0,0,0,0,...,0.810660,0.15092,0.022424,0.006173,0.009826,0.815570,0.14764,0.021164,0.005543,0.010078
1490,1,0,0,0,0,1,0,0,0,0,...,0.537290,0.30814,0.085160,0.032880,0.036533,0.537160,0.31167,0.082640,0.032250,0.036281
4551,1,0,0,0,0,1,0,0,0,0,...,0.091963,0.18430,0.151930,0.131390,0.440410,0.094734,0.18443,0.156710,0.124210,0.439910
2139,1,0,0,0,0,1,0,0,0,0,...,0.557070,0.35576,0.065004,0.014361,0.007810,0.560340,0.35248,0.063492,0.016629,0.007055
7768,1,0,0,0,0,1,0,0,0,0,...,0.639080,0.21328,0.059713,0.028345,0.059587,0.636680,0.22046,0.056941,0.028219,0.057697


GFR son un conjunto de filtros que se aplican con diferente frecuencua y orientación de forma que puede extraer las features
más importantes de una imagen como tal. Permite el analysis y segmentación de texturas.

## PHARM Dataset

In [10]:
df3 = pd.read_csv('pharm_cover.csv', header = None)
df3.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,12590,12591,12592,12593,12594,12595,12596,12597,12598,12599
6238,11192,1449,11782,1045,10643,1692,10485,1685,10841,1469,...,9395,1294,9295,1308,8881,1255,10215,1271,9019,1228
3210,7934,2850,9878,2751,6958,2713,6755,2707,8007,2366,...,10228,2039,9166,2175,9342,2241,10514,2210,9467,2184
8427,12299,2276,14221,1172,11608,2646,11108,2810,12354,2343,...,8492,2556,7089,2570,7462,2588,8298,2666,6430,2479
788,14045,840,14771,644,13919,828,13698,917,14344,903,...,15129,354,14745,576,14844,506,15423,343,14992,523
144,7160,3678,9755,3266,5971,3567,5941,3461,7201,3670,...,5029,3236,4658,2575,4501,2654,5764,3279,4022,2668


Utiliza frecuencias para extraer las features más importantes de un dataset de imagenes.

## Extracción de Features

Debido a que Kaggle ejecuta estos 3 algoritmos a la base de datos de imagenes, ya se tienen las features más relevantes para trabajar en cada uno de estos.
Ademas de esto, dentro de los datasets que ofrece kaggle, tenemos cada dataset al cual se le aplica un algoritmo para incrustar datos dentro de las imagenes, lo cual nos permite tener una mayor cantdad de data para evaluar nuestro modelo posteriormente.